## Predict if Tweet will be popular 
1. Define function which is able to predict likes and retweets
2. Add predicted like and retweet values to dic
3. Define what makes a Tweet popular 
4. Display popular Tweets 

In [1]:
import datetime
#ENTER PATH FROM DATA SOURCE!!!
DATA_SOURCE = './crwaled_data/' + str(datetime.date.today()) + '.json'

##ENTER PATH DIFFERENT FROM SOURCE!!!!
woking_data = './working_data/' +  str(datetime.date.today())

prediction_data = './predicted_data/' +  str(datetime.date.today())

In [2]:
#def function to predict values with the trained model 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.externals import joblib
import json
import pandas as pd


def merge():
    #Takes the crawld data as an input and adds the engagement_rate 
    #and saves it to working data 
    df_res = pd.read_json('./engagement_rate/engagement_rate')
    df_use = df_res[['engagement_retweets', 'engagement_likes']].copy()
    df_use = df_use.reset_index()
    df_use = df_use.rename(columns={"index": "name"})
    df_use.head(2)

    df = pd.read_json(DATA_SOURCE)
    df = pd.merge(df_use, df, on='name')

    df.to_json(woking_data, orient='records')

/home/student/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [3]:
def predict():
   
    #takes the working data as an imput and predicts the future likes and retweet counts 
    #and safes it to predicted_data 
    with open(woking_data, 'r') as myfile:
                data = json.loads(myfile.read())
            
    pf = PolynomialFeatures(degree = 2, include_bias = False)
    pf_l = PolynomialFeatures(degree = 2, include_bias = False)
    
    model = joblib.load('retweet.model')
    model_l = joblib.load('like.model')

    for i in range(len(data)):
        if data[i]['counter'] >= 4 and 'prediction_retweets' not in data[i]: 
            X_=[]
            X=[]
            X.append(data[i]['retweets'])
            X.append(data[i]['retweets 4'])
            X.append(data[i]['followers'])
            X_.append(X)
            X_test_ = pf.fit_transform(X_)
            prediction_retweets = model.predict(X_test_)[0][0]
            data[i].update(
            prediction_retweets = prediction_retweets)
            
            Y_=[]
            Y=[]
            Y.append(data[i]['likes'])
            Y.append(data[i]['likes 4'])
            Y.append(data[i]['followers'])
            Y_.append(Y)
            Y_test_ = pf_l.fit_transform(Y_)
            prediction_likes = model_l.predict(Y_test_)[0][0]
            data[i].update(
            prediction_likes = prediction_likes)
       
    
        else: 
            prediction_likes    = None 
            prediction_retweets = None
            
            data[i].update(
            prediction_retweets = prediction_retweets)
            
            data[i].update(
            prediction_likes = prediction_likes)
            
    with open(prediction_data, 'w') as fout:
        json.dump(data, fout)
### IF IT IS NOT POSSIBLE TO DUMP THE MODEL IT IS BEC. SKLEARN REMOVED THE JOBLIB LIBRARY THEN YOU NEED TO IMPORT IT
## FROM JOBLIB DIRECTLY

__EXECUDE PREDICTION__

In [4]:
# set parameters to mark a Tweet with impotant 
def important():
    with open('./predicted_data/' +  str(datetime.date.today()), 'r') as myfile:
        data = json.loads(myfile.read())

    for i in range(len(data)):
        
        if data[i]['prediction_retweets'] != None and data[i]['text'][:2] !=  'RT':
            
            if data[i]['prediction_retweets']/data[i]['followers']*100 > data[i]['engagement_retweets'] or data[i]['prediction_likes']/data[i]['followers'] *100 > data[i]['engagement_likes']:

                if data[i]['prediction_likes'] >100 or data[i]['prediction_retweets']>30:

                    important = True 

                    data[i].update( important = important )
                
                else:       
                    important = False
                    data[i].update( important = important )

            
            
            else:       
                important = False
                data[i].update( important = important )


        else:
            important = False
            data[i].update( important = important )

    with open('./predicted_data/' +  str(datetime.date.today()), 'w') as fout:
        json.dump(data, fout)


In [5]:
def display_():
    df = pd.read_json('./predicted_data/' +  str(datetime.date.today()))
    df = df[df.important ==  True]
    df = df[['name','created_at', 'followers','engagement_retweets','engagement_likes','text','prediction_retweets','prediction_likes','important'].copy()]

    display(df)

In [14]:
df = pd.read_json('./working_data/2020-01-17')
#df['prediction_retweets']

In [12]:
merge()
predict()
important()
display_()

,name,followers,engagement_retweets,engagement_likes,text,prediction_retweets,prediction_likes,important
1,ArminWolf,433183,0.000862,0.007918,Gestern wurden die ö. „Journalist*innen des Ja...,8.577105,178.054185,True
527,lou_lorenz,121255,0.002772,0.031335,Mein Großvater hat Dachau überlebt. Ich bin da...,94.290529,1628.447712,True
531,lou_lorenz,121255,0.002772,0.031335,"Jede Wette hätte ich vor einem Jahr verloren, ...",101.385788,958.425379,True
545,misik,85757,0.001044,0.010014,"PLATZ 1 DER BESTENLISTE (!)\n\nBAMM! Leute, je...",14.263637,198.731453,True


In [ ]:
import schedule
import time
schedule.every(1).minutes.do(merge)
schedule.every(1).minutes.do(predict)
schedule.every(1).minutes.do(important)
schedule.every(1).minutes.do(display_)

while True:
    schedule.run_pending()
    time.sleep(1)


,name,created_at,followers,engagement_retweets,engagement_likes,text,prediction_retweets,prediction_likes,important
0,ArminWolf,2020-02-05 14:10:55,433354,0.000862,0.007918,Das Bildungsministerium will eine Hotline zur ...,7.433011,167.386287,True
1,ArminWolf,2020-02-05 13:02:05,433354,0.000862,0.007918,"Die FDP bekam bei der Wahl in Thüringen 5,0% &...",23.246305,101.428206,True
133,MichelJungwirth,2020-02-04 15:36:46,14625,0.008739,0.042406,"Neuer Stil? Bin gespannt, wie ÖVP reagiert, we...",24.050340,107.141378,True
175,RaimundLoew,2020-02-05 06:48:27,18398,0.009751,0.030443,Gab es noch nie: Speaker zerreißt demonstrativ...,23.035799,151.100493,True
278,florianklenk,2020-02-05 14:28:36,243003,0.001332,0.010829,"Well, that escalated quickly: https://t.co/s9A...",17.609263,110.152965,True
282,florianklenk,2020-02-05 12:14:02,243003,0.001332,0.010829,"Die grüne Justizministerin, die Standesvertret...",22.658672,108.027281,True
314,lou_lorenz,2020-02-03 11:33:42,121332,0.002772,0.031335,"Warum es richtig war, vor zwölf Jahren noch ei...",2.153957,189.792638,True


,name,created_at,followers,engagement_retweets,engagement_likes,text,prediction_retweets,prediction_likes,important
0,ArminWolf,2020-02-05 14:10:55,433354,0.000862,0.007918,Das Bildungsministerium will eine Hotline zur ...,7.433011,167.386287,True
1,ArminWolf,2020-02-05 13:02:05,433354,0.000862,0.007918,"Die FDP bekam bei der Wahl in Thüringen 5,0% &...",23.246305,101.428206,True
133,MichelJungwirth,2020-02-04 15:36:46,14625,0.008739,0.042406,"Neuer Stil? Bin gespannt, wie ÖVP reagiert, we...",24.050340,107.141378,True
175,RaimundLoew,2020-02-05 06:48:27,18398,0.009751,0.030443,Gab es noch nie: Speaker zerreißt demonstrativ...,23.035799,151.100493,True
278,florianklenk,2020-02-05 14:28:36,243003,0.001332,0.010829,"Well, that escalated quickly: https://t.co/s9A...",17.609263,110.152965,True
282,florianklenk,2020-02-05 12:14:02,243003,0.001332,0.010829,"Die grüne Justizministerin, die Standesvertret...",22.658672,108.027281,True
314,lou_lorenz,2020-02-03 11:33:42,121332,0.002772,0.031335,"Warum es richtig war, vor zwölf Jahren noch ei...",2.153957,189.792638,True


,name,created_at,followers,engagement_retweets,engagement_likes,text,prediction_retweets,prediction_likes,important
0,ArminWolf,2020-02-05 14:10:55,433354,0.000862,0.007918,Das Bildungsministerium will eine Hotline zur ...,7.433011,167.386287,True
1,ArminWolf,2020-02-05 13:02:05,433354,0.000862,0.007918,"Die FDP bekam bei der Wahl in Thüringen 5,0% &...",23.246305,101.428206,True
133,MichelJungwirth,2020-02-04 15:36:46,14625,0.008739,0.042406,"Neuer Stil? Bin gespannt, wie ÖVP reagiert, we...",24.050340,107.141378,True
175,RaimundLoew,2020-02-05 06:48:27,18398,0.009751,0.030443,Gab es noch nie: Speaker zerreißt demonstrativ...,23.035799,151.100493,True
278,florianklenk,2020-02-05 14:28:36,243003,0.001332,0.010829,"Well, that escalated quickly: https://t.co/s9A...",17.609263,110.152965,True
282,florianklenk,2020-02-05 12:14:02,243003,0.001332,0.010829,"Die grüne Justizministerin, die Standesvertret...",22.658672,108.027281,True
314,lou_lorenz,2020-02-03 11:33:42,121332,0.002772,0.031335,"Warum es richtig war, vor zwölf Jahren noch ei...",2.153957,189.792638,True


,name,created_at,followers,engagement_retweets,engagement_likes,text,prediction_retweets,prediction_likes,important
0,ArminWolf,2020-02-05 14:10:55,433354,0.000862,0.007918,Das Bildungsministerium will eine Hotline zur ...,7.433011,167.386287,True
1,ArminWolf,2020-02-05 13:02:05,433354,0.000862,0.007918,"Die FDP bekam bei der Wahl in Thüringen 5,0% &...",23.246305,101.428206,True
133,MichelJungwirth,2020-02-04 15:36:46,14625,0.008739,0.042406,"Neuer Stil? Bin gespannt, wie ÖVP reagiert, we...",24.050340,107.141378,True
175,RaimundLoew,2020-02-05 06:48:27,18398,0.009751,0.030443,Gab es noch nie: Speaker zerreißt demonstrativ...,23.035799,151.100493,True
278,florianklenk,2020-02-05 14:28:36,243003,0.001332,0.010829,"Well, that escalated quickly: https://t.co/s9A...",17.609263,110.152965,True
282,florianklenk,2020-02-05 12:14:02,243003,0.001332,0.010829,"Die grüne Justizministerin, die Standesvertret...",22.658672,108.027281,True
315,lou_lorenz,2020-02-03 11:33:42,121332,0.002772,0.031335,"Warum es richtig war, vor zwölf Jahren noch ei...",2.153957,189.792638,True


,name,created_at,followers,engagement_retweets,engagement_likes,text,prediction_retweets,prediction_likes,important
0,ArminWolf,2020-02-05 14:10:55,433354,0.000862,0.007918,Das Bildungsministerium will eine Hotline zur ...,7.433011,167.386287,True
1,ArminWolf,2020-02-05 13:02:05,433354,0.000862,0.007918,"Die FDP bekam bei der Wahl in Thüringen 5,0% &...",23.246305,101.428206,True
133,MichelJungwirth,2020-02-04 15:36:46,14625,0.008739,0.042406,"Neuer Stil? Bin gespannt, wie ÖVP reagiert, we...",24.050340,107.141378,True
175,RaimundLoew,2020-02-05 06:48:27,18398,0.009751,0.030443,Gab es noch nie: Speaker zerreißt demonstrativ...,23.035799,151.100493,True
278,florianklenk,2020-02-05 14:28:36,243003,0.001332,0.010829,"Well, that escalated quickly: https://t.co/s9A...",17.609263,110.152965,True
282,florianklenk,2020-02-05 12:14:02,243003,0.001332,0.010829,"Die grüne Justizministerin, die Standesvertret...",22.658672,108.027281,True
315,lou_lorenz,2020-02-03 11:33:42,121332,0.002772,0.031335,"Warum es richtig war, vor zwölf Jahren noch ei...",2.153957,189.792638,True


...

,name,created_at,followers,engagement_retweets,engagement_likes,text,prediction_retweets,prediction_likes,important
0,ArminWolf,2020-02-05 14:10:55,433354,0.000862,0.007918,Das Bildungsministerium will eine Hotline zur ...,7.433011,167.386287,True
1,ArminWolf,2020-02-05 13:02:05,433354,0.000862,0.007918,"Die FDP bekam bei der Wahl in Thüringen 5,0% &...",23.246305,101.428206,True
13,ArminWolf,2020-02-05 16:15:41,433356,0.000862,0.007918,Der belgische Liberale Giy Verhofstadt ist von...,22.239494,156.323520,True
216,MichelJungwirth,2020-02-04 15:36:46,14625,0.008739,0.042406,"Neuer Stil? Bin gespannt, wie ÖVP reagiert, we...",24.050340,107.141378,True
262,RaimundLoew,2020-02-05 06:48:27,18398,0.009751,0.030443,Gab es noch nie: Speaker zerreißt demonstrativ...,23.035799,151.100493,True
418,florianklenk,2020-02-05 14:28:36,243003,0.001332,0.010829,"Well, that escalated quickly: https://t.co/s9A...",17.609263,110.152965,True
422,florianklenk,2020-02-05 12:14:02,243003,0.001332,0.010829,"Die grüne Justizministerin, die Standesvertret...",22.658672,108.027281,True
489,lou_lorenz,2020-02-03 11:33:42,121332,0.002772,0.031335,"Warum es richtig war, vor zwölf Jahren noch ei...",2.153957,189.792638,True


,name,created_at,followers,engagement_retweets,engagement_likes,text,prediction_retweets,prediction_likes,important
0,ArminWolf,2020-02-05 14:10:55,433354,0.000862,0.007918,Das Bildungsministerium will eine Hotline zur ...,7.433011,167.386287,True
1,ArminWolf,2020-02-05 13:02:05,433354,0.000862,0.007918,"Die FDP bekam bei der Wahl in Thüringen 5,0% &...",23.246305,101.428206,True
13,ArminWolf,2020-02-05 16:15:41,433356,0.000862,0.007918,Der belgische Liberale Giy Verhofstadt ist von...,22.239494,156.323520,True
216,MichelJungwirth,2020-02-04 15:36:46,14625,0.008739,0.042406,"Neuer Stil? Bin gespannt, wie ÖVP reagiert, we...",24.050340,107.141378,True
262,RaimundLoew,2020-02-05 06:48:27,18398,0.009751,0.030443,Gab es noch nie: Speaker zerreißt demonstrativ...,23.035799,151.100493,True
418,florianklenk,2020-02-05 14:28:36,243003,0.001332,0.010829,"Well, that escalated quickly: https://t.co/s9A...",17.609263,110.152965,True
422,florianklenk,2020-02-05 12:14:02,243003,0.001332,0.010829,"Die grüne Justizministerin, die Standesvertret...",22.658672,108.027281,True
489,lou_lorenz,2020-02-03 11:33:42,121332,0.002772,0.031335,"Warum es richtig war, vor zwölf Jahren noch ei...",2.153957,189.792638,True


,name,created_at,followers,engagement_retweets,engagement_likes,text,prediction_retweets,prediction_likes,important
0,ArminWolf,2020-02-05 14:10:55,433354,0.000862,0.007918,Das Bildungsministerium will eine Hotline zur ...,7.433011,167.386287,True
1,ArminWolf,2020-02-05 13:02:05,433354,0.000862,0.007918,"Die FDP bekam bei der Wahl in Thüringen 5,0% &...",23.246305,101.428206,True
13,ArminWolf,2020-02-05 16:15:41,433356,0.000862,0.007918,Der belgische Liberale Giy Verhofstadt ist von...,22.239494,156.323520,True
216,MichelJungwirth,2020-02-04 15:36:46,14625,0.008739,0.042406,"Neuer Stil? Bin gespannt, wie ÖVP reagiert, we...",24.050340,107.141378,True
262,RaimundLoew,2020-02-05 06:48:27,18398,0.009751,0.030443,Gab es noch nie: Speaker zerreißt demonstrativ...,23.035799,151.100493,True
418,florianklenk,2020-02-05 14:28:36,243003,0.001332,0.010829,"Well, that escalated quickly: https://t.co/s9A...",17.609263,110.152965,True
422,florianklenk,2020-02-05 12:14:02,243003,0.001332,0.010829,"Die grüne Justizministerin, die Standesvertret...",22.658672,108.027281,True
489,lou_lorenz,2020-02-03 11:33:42,121332,0.002772,0.031335,"Warum es richtig war, vor zwölf Jahren noch ei...",2.153957,189.792638,True


,name,created_at,followers,engagement_retweets,engagement_likes,text,prediction_retweets,prediction_likes,important
0,ArminWolf,2020-02-05 14:10:55,433354,0.000862,0.007918,Das Bildungsministerium will eine Hotline zur ...,7.433011,167.386287,True
1,ArminWolf,2020-02-05 13:02:05,433354,0.000862,0.007918,"Die FDP bekam bei der Wahl in Thüringen 5,0% &...",23.246305,101.428206,True
13,ArminWolf,2020-02-05 16:15:41,433356,0.000862,0.007918,Der belgische Liberale Giy Verhofstadt ist von...,22.239494,156.323520,True
216,MichelJungwirth,2020-02-04 15:36:46,14625,0.008739,0.042406,"Neuer Stil? Bin gespannt, wie ÖVP reagiert, we...",24.050340,107.141378,True
262,RaimundLoew,2020-02-05 06:48:27,18398,0.009751,0.030443,Gab es noch nie: Speaker zerreißt demonstrativ...,23.035799,151.100493,True
418,florianklenk,2020-02-05 14:28:36,243003,0.001332,0.010829,"Well, that escalated quickly: https://t.co/s9A...",17.609263,110.152965,True
422,florianklenk,2020-02-05 12:14:02,243003,0.001332,0.010829,"Die grüne Justizministerin, die Standesvertret...",22.658672,108.027281,True
489,lou_lorenz,2020-02-03 11:33:42,121332,0.002772,0.031335,"Warum es richtig war, vor zwölf Jahren noch ei...",2.153957,189.792638,True


,name,created_at,followers,engagement_retweets,engagement_likes,text,prediction_retweets,prediction_likes,important
0,ArminWolf,2020-02-05 14:10:55,433354,0.000862,0.007918,Das Bildungsministerium will eine Hotline zur ...,7.433011,167.386287,True
1,ArminWolf,2020-02-05 13:02:05,433354,0.000862,0.007918,"Die FDP bekam bei der Wahl in Thüringen 5,0% &...",23.246305,101.428206,True
13,ArminWolf,2020-02-05 16:15:41,433356,0.000862,0.007918,Der belgische Liberale Giy Verhofstadt ist von...,22.239494,156.323520,True
216,MichelJungwirth,2020-02-04 15:36:46,14625,0.008739,0.042406,"Neuer Stil? Bin gespannt, wie ÖVP reagiert, we...",24.050340,107.141378,True
262,RaimundLoew,2020-02-05 06:48:27,18398,0.009751,0.030443,Gab es noch nie: Speaker zerreißt demonstrativ...,23.035799,151.100493,True
418,florianklenk,2020-02-05 14:28:36,243003,0.001332,0.010829,"Well, that escalated quickly: https://t.co/s9A...",17.609263,110.152965,True
422,florianklenk,2020-02-05 12:14:02,243003,0.001332,0.010829,"Die grüne Justizministerin, die Standesvertret...",22.658672,108.027281,True
489,lou_lorenz,2020-02-03 11:33:42,121332,0.002772,0.031335,"Warum es richtig war, vor zwölf Jahren noch ei...",2.153957,189.792638,True
